In [ ]:
!pip install flask-ngrok
!pip install gevent

     |████████████████████████████████| 5.8 MB 9.2 MB/s 
     |████████████████████████████████| 251 kB 53.3 MB/s 


In [ ]:
from __future__ import division, print_function
# coding=utf-8
import sys
import os
import glob
import re
import numpy as np

# Keras
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50

# Flask utils
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

from flask import Flask, redirect, url_for, request, render_template
from flask_ngrok import run_with_ngrok

# Define a flask app
app = Flask(__name__, template_folder='drive/MyDrive/Colab Notebooks/templates') #change directory
# run_with_ngrok(app)   

# Load your trained model
model = ResNet50(weights='imagenet')

# Optional - print status update
print('Model loaded.')


def model_predict(img_path, model):
    img = image.load_img(img_path, target_size=(224, 224))

    # Preprocessing the image
    x = image.img_to_array(img)
    # x = np.true_divide(x, 255)
    x = np.expand_dims(x, axis=0)

    # Caffe is a is a deep learning framework made with speed in mind.
    # Caffe is developed by Berkeley AI Research (BAIR) and by community contributors.
    x = preprocess_input(x, mode='caffe')

    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.path.dirname(os.path.abspath("__file__"))
        file_path = os.path.join(
            'drive/MyDrive/Colab Notebooks/uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)

        # Process your result for user
        # decode_predictions(preds, top=1) returns a list of lists of top class prediction tuples
        pred_class = decode_predictions(preds, top=1)   # ImageNet Decode
        result = str(pred_class[0][0][1])               # Convert to string
        return render_template('predict.html', result=result)
    return None


if __name__ == "__main__":
    app.run()


https://9o5n7yq10eg-496ff2e9c6d22116-5000-colab.googleusercontent.com/
102981632/102967424 [==============================] - 1s 0us/step
Model loaded.
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [23/Aug/2021 02:01:01] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 02:01:02] "GET /favicon.ico HTTP/1.1" 404 -


49152/35363 [=========================================] - 0s 0us/step


127.0.0.1 - - [23/Aug/2021 02:01:28] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 02:01:29] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Aug/2021 02:19:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 02:19:08] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Aug/2021 02:19:11] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 02:19:12] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [23/Aug/2021 02:19:21] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [23/Aug/2021 02:19:21] "GET /favicon.ico HTTP/1.1" 404 -
